In [ ]:
%%writefile modules/data_utils.py

import os
import glob

def get_sample_number(sample_dir): #eg:  '/content/dataset/test/blink/100/10/02685.bmp' -> 2685
  return int(sample_dir.split('/')[-1][:-4])

def img_lst(file_path):
  file_lst = []
  if 'unblink' in file_path: y = 1
  else: y = 0

  for file in os.listdir(file_path):
    if file.endswith('.bmp') : file_lst.append(f'{file_path}/{file}')
  return sorted(file_lst, key= get_sample_number), y

def get_dataset_filedirvslabel(data_dir):
  dataset=[]
  for data_dir_subject in glob.glob(f'{data_dir}/*'):
    if not os.path.isdir(data_dir_subject):continue
    dataset.append(img_lst(f'{data_dir_subject}/10'))
  return dataset

In [ ]:
%%writefile modules/feature_functions_utils.py


from skimage.draw import line
import numpy as np
import cv2

def get_eye_line(eye_landmarks):
    l0, l1, l2, l3, l4, l5 = eye_landmarks.astype('int')
    A= list(((np.array(l1) + np.array(l5))/2).astype('int'))
    B= list(((np.array(l2) + np.array(l4))/2).astype('int'))

    
    line_ = list(zip(*line(*l0, *A))) + list(zip(*line(*A, *B))) + list(zip(*line(*B, *l3)))

    return line_

def get_frp(image_gray, landmarks, return_img= False):
    min_pixel_color= 255
    max_pixel_color=0

    eye_line = get_eye_line(landmarks)

    for x,y in eye_line:
        gray_color = image_gray[y, x]

        if gray_color>max_pixel_color:
            max_pixel_color=gray_color
            y_max, x_max= [y, x]
        if gray_color<min_pixel_color:
            min_pixel_color=gray_color
            y_min, x_min= [y, x]

    for x,y in eye_line:
        image_gray[y, x]=255
    image_gray[y_min-2:y_min+2, x_min-2:x_min+2]=0
    image_gray[y_max-2:y_max+2, x_max-2:x_max+2]=0

    frp= (max_pixel_color+0.1)/(min_pixel_color+0.1)
    if return_img:return image_gray, frp
    else:return frp

def img2eye(image_gray, eye_landmark, landmark=None, resize= 28):
  eye_middle = list(map(int, np.mean(eye_landmark, axis=0)))
  eye_width = int((landmark[9][1] - landmark[28][1])/5)
  eye_image = image_gray[eye_middle[1]-eye_width: eye_middle[1]+eye_width, eye_middle[0]-eye_width: eye_middle[0]+eye_width]
  return cv2.resize(eye_image, (resize, resize))


In [ ]:
%%writefile modules/feature_functions.py

from modules.feature_functions_utils import *

def feature_func_eyeimages(image_gray, eye1_landmark, eye2_landmark, all_landmarks):
  resize= 28
  eye1_img = img2eye(image_gray, eye1_landmark, all_landmarks, resize= 28)
  eye2_img = img2eye(image_gray, eye2_landmark, all_landmarks, resize= 28)

  return np.array([eye1_img, eye2_img])

def feature_func_frp(image_gray, eye1_landmark, eye2_landmark, all_landmarks):
  eye1_frp = get_frp(image_gray, eye1_landmark, return_img= False)
  eye2_frp = get_frp(image_gray, eye1_landmark, return_img= False)

  return np.array([eye1_frp, eye2_frp])

In [ ]:
%%writefile modules/get_features.py
import cv2
import numpy as np

def image2features(image_gray, feature_func, faces, landmark_detector=None): #features can be eye images/ frp-like features
  landmark = landmark_detector.fit(image_gray, faces)[1][0][0] #landmarks of first person
  eye1_landmark= landmark[36:42]
  eye2_landmark= landmark[42:48]

  features= feature_func(image_gray, eye1_landmark, eye2_landmark, landmark) # return 2 features for both eyes
  return features

def get_features_for_sample(dataset_filedirvslabel, sample_id, feature_func, detector=None, landmark_detector=None):
  vid_frame_dirs, label = dataset_filedirvslabel[sample_id]

  features_eye1, features_eye2=[], []
  for img_dir in vid_frame_dirs:
    image = cv2.imread(img_dir)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(image_gray)

    if len(faces)!=0:
      feature_eye1, feature_eye2 = image2features(image_gray, feature_func, faces, landmark_detector)

      features_eye1.append(feature_eye1)
      features_eye2.append(feature_eye2)

    else:
      landmark= 'no face'
      print(f"no faces : {img_dir}")
      return None, None
  return np.array([features_eye1, features_eye2]), label

In [ ]:
%%writefile modules/detectors.py
import urllib.request as urlreq
import cv2
import matplotlib.pyplot as plt
import numpy as np

def get_detectors(detector_name = 'haarcascade'):
  if detector_name == 'haarcascade':
    # https://github.com/Danotsonof/facial-landmark-detection
    haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml"
    haarcascade = "haarcascade_frontalface_alt2.xml"
    urlreq.urlretrieve(haarcascade_url, haarcascade)
    detector = cv2.CascadeClassifier(haarcascade)

    LBFmodel_url = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"
    LBFmodel = "LFBmodel.yaml"
    urlreq.urlretrieve(LBFmodel_url, LBFmodel)
    landmark_detector  = cv2.face.createFacemarkLBF()
    landmark_detector.loadModel(LBFmodel)

    return detector, landmark_detector
  else:
    raise NotImplementedError